**Импорт библиотек:**

In [ ]:
import pandas as pd

import missingno as msno

import re

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

import seaborn as sns

import phik

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import TargetEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

from xgboost import XGBRegressor

import numpy as np

from lightgbm import LGBMRegressor

from sklearn.svm import SVR

import warnings
warnings.filterwarnings('ignore')

import time

from catboost import CatBoostRegressor

pd.set_option('display.max_colwidth', None)

## Подготовка данных

### Загрузка и предпросмотр

In [ ]:
 try:
        df = pd.read_csv('autos.csv', parse_dates=['DateCrawled'])
 except FileNotFoundError:
        df = pd.read_csv('datasets/autos.csv', parse_dates=['DateCrawled'])
df.head(2)

Визуально таблицу видим, рассмотрим ее подробнее, чтобы сделать первые выводы. Для этого используем функцию, которая сразу покажет всю, интересующую нас информацию о строении таблицы:

In [ ]:
# функция для ознакомления с таблицами
def observe_data(df):
    print('Превью таблицы (первые пять строк):\n')
    display(df.head())
    print('=' * 90)
    print('Информация о столбцах и типах данных:\n')
    df.info()
    print('=' * 90)
    print('Статистическая информация:\n')
    display(df.describe().T)

In [ ]:
observe_data(df)

Первые выводы о датасете:
+ Исходный датасет имеет 354369 наблюдений по 16 признакам. 
+ Несоответствия количества строк по признакам говорит нам о наличии пропусков. 
+ Типов данных всего два: int64 и object. В таблице есть признаки с явным указанием на даты, такие как `DateCrawled` или `LastSeen`, значит есть проблемы и с несоответствием типов данных. 
+ В статистической информации мы видим наличие аномальных нулевых значение в таких столбцах как `Price` или `Power`. 
+ Дополнительно можно сказать об отклонение от общепринятых стандартах в названиях столбцов.

Покажем визуально пропуски датасета:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
msno.bar(df, ax=ax, fontsize=12, sort="ascending", color="tab:blue")
ax.set_title('Диаграмма пропусков', fontsize=24, color='blue', pad=20)
ax.set_xlabel('Признаки', fontsize=14, labelpad=20)
ax.set_ylabel('Количество наблюдений', fontsize=14, labelpad=20)
plt.show()

Всех больше пропусков имеет признак `Repaired` - 71154. 

### Предобработка данных

#### Неверный тип данных

Проблему с неправильным определением типа *datetime* решим на этапе парсирования данных:

In [ ]:
date_cols = ['DateCrawled', 'DateCreated', 'LastSeen']
try:
        df = pd.read_csv('autos.csv', parse_dates=date_cols)
except FileNotFoundError:
        df = pd.read_csv('datasets/autos.csv', parse_dates=date_cols)
df.info()

Дата теперь стала датой. Посмотрим на имена столбцов.

#### Переименование столбцов

In [ ]:
df.columns

Нам нужно все буквы сделать строчными, а перед каждой второй и последующими заглавными поставить символ подчеркивания. Применим регулярные выражения:

In [ ]:
df.columns = [re.sub(r'(?<!^)(?=[A-Z])', '_', name).lower() for name in df.columns]
df.columns

#### Удаление неинформативных признаков

Перед дальнейшей работой разберемся со столбцом `number_of_pictures`. Статистика указала, что все значения нулевые.

In [ ]:
df['number_of_pictures'].unique()

Это константный признак говорит о том, что все объекты не имеют фотографий, не является информативным для нас. Удалим его.

In [ ]:
df.drop('number_of_pictures', axis=1, inplace=True)

#### Устранение пропусков

Теперь можно вернуться к проблеме с пропусками:

In [ ]:
df.isna().sum()

Все признаки с пропусками имеют строковую информацию. Причем информация о кузове, модели авто, наличия факта ремонта является определяющей для цены. Для начала поставим заглушку `unknown` на месте пропусков признака `repaired`. Остальные признаки заполним наиболее часто встречаемыми значениями в категории, в противном случае тоже поставим заглушку.

In [ ]:
# missing_cols = df.columns[df.isna().sum() > 0].tolist()
# df[missing_cols] = df[missing_cols].fillna('unknown')
# df.isna().sum()

In [ ]:
missing_cols = df.columns[df.isna().sum() > 0].tolist()
missing_cols

In [ ]:
df['repaired'] = df['repaired'].fillna('unknown')

In [ ]:
df['model'] = df.groupby('brand')['model'].transform(
    lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'unknown')
)
df['model'][df['model'] == 'unknown'].count()

In [ ]:
for column in df.columns[df.isna().sum() > 0].tolist():
    
    df[column] = df.groupby('model')[column].transform(
    lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'unknown')
)

#### Устранение дубликатов

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace=True)
df.duplicated().sum()


В этом подразделе исправили следующие проблемы:
+ считывание дат, теперь они имеют тип *datetime*;
+ названия столбцов поменяли на корректные;
+ все пропуски заменили на заглушку;
+ устранили явные дубликаты.

Проверку на неявные дубликаты и выявление и устранение аномальных значение проведем попутно с исследованием данных.

### Исследовательский анализ

Все данные можно разделить на количественные, категориальные и временные. Рассмотрим количественные (включим в рассмотрение и категориальные столбцы `registration_year`, `postal_code`):

#### Количественные данные

In [ ]:
num_cols = df.select_dtypes(include='int').drop(['registration_month'], axis=1).columns
num_cols

Рассмотрим и удалим сначала все аномалии, потом построим гистограмму распределения и диаграмму размаха количественных признаков.

In [ ]:
df['price'].describe()

In [ ]:
df[df['price'] == 0].shape

Свыше 10000 автомобилей судя по таблице раздают бесплатно, нужно отфильтровать этот столбец.

In [ ]:
df['price'].hist(bins=50, edgecolor='black', grid=True)
plt.xlabel('Цена, евро')
plt.ylabel('Количество наблюдений')
plt.title('Гистограмма распределения цены автомобилей')
plt.show()

Подробней рассмотрим цену вблизи нуля, чтобы определить порог фильтра.

In [ ]:
df['price'].hist(bins=1000, edgecolor='black', grid=True)
plt.xlim(0, 200)
plt.xlabel('Цена, евро')
plt.ylabel('Количество наблюдений')
plt.title('Гистограмма распределения цены автомобилей')
plt.show()

In [ ]:
df.loc[df['price'] < 25].head()

Слабо верится, что мерседес можно приобрести за 1 евро. Отфильтруем машины по цены, превышающие 25 евро, т.к. до этой цены мы наблюдаем 

In [ ]:
df = df.loc[df['price'] > 25]

In [ ]:
df.shape

Теперь рассмотрим год регистрации авто.

In [ ]:
df['registration_year'].describe()

Здесь тоже что-то не в порядке с годами, есть машины из далекого прошлого или будущего.

In [ ]:
df['registration_year'].hist(bins=50, edgecolor='black', grid=True)
plt.xlabel('Год регистрации автомобиля')
plt.ylabel('Количество наблюдений')
plt.title('Гистограмма распределения года регистрации автомобиля');

In [ ]:
df['registration_year'].hist(bins=1000, edgecolor='black', grid=True)
plt.xlim(1900, 1980)
plt.ylim(0, 100)
plt.xlabel('Год регистрации автомобиля')
plt.ylabel('Количество наблюдений')
plt.title('Гистограмма распределения года регистрации автомобиля');

In [ ]:
df['last_seen'].max()

Видим, что превышать хотя бы 100 единиц, начали машины, зарегестрированные с середины 50-х. Возьмем 1955 за нижнюю границу. А верхней сделаем 2016 год, как максимальный год из временного признака `last_seen`.

In [ ]:
df = df.loc[(df['registration_year'] > 1955) & (df['registration_year'] < 2017)]

In [ ]:
df.shape

Перейдем к признаку, характеризующему мощность.

In [ ]:
df['power'].describe()

Машины слабее лошади с одной стороны и сопоставимые по мощности с ракетным двигателем с другой. Явно что-то не так. Отфильтруем этот признак следующим образом: примем, что старые машины могут иметь мощность 10 л.с., а суперкары 2016 года на превышали 2000.

In [ ]:
df.loc[(df['power'] < 10) | (df['power'] > 2000)].shape

In [ ]:
df['power'] = df.groupby('model')['power'].transform(
    lambda x: x.where(((x > 10) & (x < 2000)), x.mode()[0] if not x.mode().empty else x)
)

In [ ]:
df.loc[(df['power'] < 10) | (df['power'] > 2000)].shape

Удалось заполнить модой почти 28000 значений. Оставшиеся отфильтруем.

In [ ]:
df['power'].describe()

In [ ]:
df = df.loc[(df['power'] > 10) & (df['power'] < 2000)]

In [ ]:
df.shape

In [ ]:
df['kilometer'].describe()

Здесь все выглядит правдоподобно, ничего фильтровать не будем.

In [ ]:
df.shape

Построим граифики количественных признаков. Для этого напишем функцию для построения каждого типа графика:

In [ ]:
def make_histogramm(ax, df, column):
    
    ax.hist(x=df[column], bins=30, edgecolor='black')
    plt.grid(color='grey', linestyle='--', linewidth=0.5)
    ax.set_title(f'Гистограмма распределения признака {column}')
    ax.set_xlabel('Значение признака')
    ax.set_ylabel('Количество наблюдений')

In [ ]:
def make_boxplot(ax, df, column):
    
    ax.boxplot(df[column], vert=False)
    plt.grid(color='grey', linestyle='--', linewidth=0.5)
    ax.set_title(f'Диаграмма размаха признака {column}')
    ax.set_xlabel('Значение признака')

In [ ]:
for column in num_cols:
    
    fig = plt.figure(figsize=(15, 5))
    gs = GridSpec(nrows=2, ncols=1, figure=fig)
    plt.suptitle(f'Графики признака {column}', va='center', fontsize=14)
    

        
    ax = fig.add_subplot(gs[0,0])
    make_histogramm(ax, df, column)
    ax = fig.add_subplot(gs[1,0])
    make_boxplot(ax, df, column)

    plt.tight_layout()
    plt.show()

Посмотрим на тип связи между таргетом и количественными признаками:

In [ ]:
sns.pairplot(df, vars=num_cols)
plt.suptitle(f'Графики рассеяния', y=1.02, fontsize=16)
plt.show()

Целевой признак не имеет линейной связи ни с одним из признаков. 

Резюме по количественным данным:
+ Признак `price` распределен не нормально, имеет выбросы.
+ `registration_year` нормально распределен, имеет выбросы. В среднем представлены автомобили, зарегистрированные после 2000 года.
+ `power` нормально распределен со смещением, есть выбросы.
+ `kilometer` большая часть автомобилей имеет пробег более 140_000 км.
+ `postal_code` географически автомобили распределены нормально.
+ Признаки между собой связаны нелинейно.

#### Категориальные данные

Здесь мы также в начале избавимся от аномалий, а затем построим диаграммы для лучшего понимания данных.

In [ ]:
cat_cols = df.drop(num_cols, axis=1).select_dtypes(exclude='datetime64').columns
cat_cols

In [ ]:
df['vehicle_type'].value_counts()

Не все категории этого признака семантически равнозначны, так, например, непонятно, какой тип кузова представляет категория `other`. 

In [ ]:
df['gearbox'].unique()

In [ ]:
df['model'].unique()

Неявных дубликатов здесь не наблюдаем.

In [ ]:
df['registration_month'].unique()

Вероятно какой-то программист-автовладелец за январь поставил 0 месяц. Исправим так, чтобы месяцы были от 1 до 12.

In [ ]:
df.loc[df['registration_month'] == 0, 'registration_month'] = 1

In [ ]:
df['fuel_type'].unique()

Видим наименование категорий `petrol` и `gasoline`, что является одним и тем же видом топлива.

In [ ]:
df.loc[df['fuel_type'] == 'petrol', 'fuel_type'] = 'gasoline'

In [ ]:
df['brand'].unique()

In [ ]:
df['repaired'].unique()

Неявные дубликаты были выявлены только в столбце `fuel_type`.

Построим диаграммы распределений для наибольших категорий признаков `vehicle_type`, `model`, `brand`:

In [ ]:
for column in ['vehicle_type', 'model', 'brand']:
    (df[column]
    .value_counts()
    .head(10)
    .sort_values()
    .plot(kind='barh', figsize=(10, 5), edgecolor='black', grid=True)
     )
    
    plt.title(f'Топ категорий в распределение признака {column}')
    plt.ylabel(f'{column}')
    plt.xlabel('Количество')
    plt.show()

В этих же признаках оценим наиболее дорогие (медианные) автомобили: 

In [ ]:
for column in ['vehicle_type', 'model', 'brand']:    
    (df.groupby([column])
         .agg({'price': 'median'})
         .head(10)
         .sort_values(by='price')
         .plot(kind='barh', figsize=(10, 5), edgecolor='black', grid=True)
    )
    plt.title(f'Топ категорий по медианной цене признака {column}')
    plt.ylabel('Категория')
    plt.xlabel('Медианная цена, евро')
    plt.grid(True)
    plt.show()

+ Наиболее востребованными типами кузова - седан, маленький городской, универсал. Наиболее дорогой - внедорожники, кабриолеты.
+ Среди моделей самые распространненые - гольф и бмв третьей модели. Наиболее дорогие: крайслер (300c), бмв(1er) и альфа-ромео(159).
+ Из марок автомобиля было продано больше всех - фольксваген, опель и бмв. Из которых наиболее дорогие по медиане - бмв, ауди и дация.

Построим круговые диаграммы для оставшихся признаков:

In [ ]:
for column in ['gearbox', 'fuel_type', 'repaired']:
    plt.figure(figsize=(7, 7))
    values = df[column].value_counts().values
    labels = df[column].value_counts().index
    plt.pie(
        values,
        wedgeprops={'edgecolor': 'w'},
        shadow=True,
        pctdistance = 0.5,
        autopct=lambda x: f'{x:.1f}%' if x > 1 else ''
    )
    plt.legend(labels=labels, loc=[1.01, 0.5])
    plt.title(f'Диаграмма признака {column}', fontsize=16)
    plt.show()


Также посмотрим, сколько стоят разные категории рассмотренных категориальных признаков. Цены будем рассматривать снова медианные.

In [ ]:
for column in ['gearbox', 'fuel_type', 'repaired']:
    
    index = (df
             .pivot_table(index=column, values='price', aggfunc='median')
             .sort_values(by='price', ascending=False)
             .index)
    values = (df
        .pivot_table(index=column, values='price', aggfunc='median')
        .sort_values(by='price', ascending=False)['price']
        .tolist())
    
    plt.figure(figsize=(8, 6))
    ax = plt.bar(index, values, width=0.3, edgecolor='black')
    plt.title(f'Медианная цена категорий признака {column}', fontsize=14)
    plt.xlabel('Категории')
    plt.ylabel('Цена, евро')
    plt.grid(True)
    plt.show()

+ Самым распространненым типом коробки передач является механическая, а дорогая автоматическая, что весьма логично.
+ С большим отрывом лидируют бензиновые автомобили благодаря нижней границе года регистрации авто, когда это был практически единственный вариант. Самыми дорогими являются современные технологичные гибридные и электромобили.
+ На вторичном рынке автоболей в большинстве представлены авто, не бывавшие в ремонте, они соответсвенно дороже.

Перед корреляционным анализом исключим все временные данные, месяц регистрации, т.к. они не являются информативными для нашей задачи. Также преобразуем год регистрации в возраст автомобиля для упрощения логики этого признака и сократим номер почтового индекса для уменьшений количества категорий признака.

In [ ]:
cur_year = df['date_crawled'].dt.year.max()
df = df.rename(columns={'registration_year': 'age'})
df['age'] = cur_year - df['age']

In [ ]:
len(df['postal_code'].unique())

In [ ]:
df['reg_postal'] = df['postal_code'] // 100
df['reg_postal'] = df['reg_postal'].astype('object')

In [ ]:
len(df['reg_postal'].unique())

In [ ]:
df = df.drop(['date_crawled', 'date_created', 'last_seen', 'postal_code', 'registration_month'], axis=1)

In [ ]:
df.head(2)

Проверим изменятся ли коэффициенты корреляции с таргетом входных признаков, если мы уберем из датасета все строки с заглушкой *unknown*.

In [ ]:
figsize = (10, 10)
fig = plt.subplots(figsize=figsize)
plt.title('Phik корреляция')
sns.heatmap(df
            .phik_matrix(interval_cols=num_cols), 
            annot=True, cmap='cividis', 
            linewidth=0.1,
            fmt='.2f',
            square=True)
plt.show()

+ Самую высокую корреляцию имеют тип кузова и модель автомобиля(0.9) и модель и марка(1), которые являются взаимозаменяемыми признаками, потому что каждая марка имеет уникальные модели. При подготовке данных для обучения модели лучше исключить марку, т.к. она меньше коррелирует с таргетом чем модель.
+ С целевым признаком больше всего коррелирует год регистрации авто(0.66), что отражает возраст, и модель автомобиля(0.58).
+ Корреляция меняется незначительно, если убрать строки с заглушкой, которую поставили при заполнении пропусков. Это означает, что можно ее оставить в тренировочной и тестовой выборке.

**Вывод:**

В этом разделе мы познакомились с предоставленными данными: изначально датасет имеет свыше 350 тыс. строк по 16 признакам, включая целевой `price`. Датасет имеет следующие проблемы: большое количество пропусков, неверный тип данных при считывании таблицы, аномальные значения признаков.

В ходе предобработки эти проблемы были устранены. Пропуски, замененные на маркер *unknown*  мы так оставим, т.к. эти категориальные признаки не имеют сильной корреляции с таргетом. Кроме этого сравнив два датасета с удаленными строками и просто с заглушкой обнаружили, что коэффициенты корреляции меняются незначительно. Аномальные значения были устранены путем фильтра на соответствующие признаки. Также удалили малозначимые для прогноза временные признаки и малоинформативный признак `NumberOfPictures`.

Среди количественных признаков наш целевой имеет распределение, отличное от нормального. Также отметим наличие выбросов для всех признаков. Признаки имеют нелинейную связь.

Однако судя по коэффициентам корреляции наибольшую связь с таргетом имеют категориальные признаки `age` и `model`, но даже между этими признаками эта связь несильная. Чего не скажешь про `model` и `brand` наличие которых указывает на частичную мултиколлинеарность и будет способстовать понижению эффективности модели линейной регрессии, но для дерева решений это не является проблемой. 




## Обучение моделей

Использовав полученную выше информацию, подготовим выборки для обучения и тестирования моделей. В качестве моделей будем выбирать из ансамблиевых моделей XGBoost, LightGBM, CatBoost, в качестве базовой рассмотрим TreeRegressor. Для более удобного способа подбора используем PipiLine.

### Подготовка выборки для обучения

Разделим данные на целевой(y) тренировочный/валидационный/тестовый и входные признаки(x).

In [ ]:
RANDOM_STATE = 13

y = df['price']
X = df.drop(['price', 'brand'], axis=1) 

X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y,
    test_size=0.4,
    random_state=RANDOM_STATE
)

X_valid, X_test, y_valid, y_test = train_test_split(
    X_test, 
    y_test,
    test_size=0.5,
    random_state=RANDOM_STATE
)

Снова разделим данные на количественные и категориальные, т.к. у нас произошли исключения признаков:


In [ ]:
num_columns = ['age', 'power', 'kilometer']
cat_columns = ['vehicle_type', 
               'gearbox', 
               'model', 
               'fuel_type', 
               'reg_postal', 
               'repaired']

Создадим Pipiline, который проведет стандартизацию для количественных данных и кодирование для категориальных.

In [ ]:
data_preprocessor = ColumnTransformer(
    [
        ('target_enc', TargetEncoder(target_type="continuous"), cat_columns),
        ("numerical", "passthrough", num_columns)
        
    ], 
    remainder='passthrough'
)

pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', DecisionTreeRegressor(random_state=RANDOM_STATE))
])

### Подбор моделей

В качестве baseline модели будем использовать простое дерево решений, другими моделями будут LGBM и CatBoost, основанные на ансамбле деревьев с использованием градиентного бустинга.

Подберем гиперпараметры для первой модели дерева решений. Будем смотреть на топ-5 моделей, чтобы видеть изменение гиперпараметров в лучших моделях:

In [ ]:
param_grid = [
    {
        'models': [DecisionTreeRegressor(random_state=RANDOM_STATE)],
        'models__max_depth': range(2, 100),
        'models__max_features': range(2, 100),
        'models__min_samples_split': range(2, 100)
    }
]

start = time.time()

tree_descision_model = RandomizedSearchCV(
    pipe_final,
    param_distributions=param_grid, 
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    random_state=RANDOM_STATE
)

tree_descision_model.fit(X_train, y_train)

selection_time = round(time.time() - start, 5)
print(f'Продолжительность подбора модели {selection_time} c.')

rmse_train_tree = tree_descision_model.best_score_
print('Метрика RMSE для лучшей модели:\n', -rmse_train_tree)


result = pd.DataFrame(tree_descision_model.cv_results_).sort_values('rank_test_score')
result[['rank_test_score', 'param_models', 'mean_test_score','params']].head(5)

Оставим модель с этими гиперпараметрами и преступим к подбору модели LGBMRegressor:

In [ ]:
param_grid = [
    {
        'models': [LGBMRegressor(verbosity=-1)],
        'models__learning_rate': [0.1, 0.5, 0.8],  
        'models__n_estimators': [10, 50, 100], 
        'models__num_leaves': [31, 40, 50], 
        'models__max_depth': [3, 5, 15]
    }
]

start = time.time()

lgbm_model = RandomizedSearchCV(
    pipe_final,
    param_distributions=param_grid, 
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    random_state=RANDOM_STATE
)

lgbm_model.fit(X_train, y_train)

selection_time = round(time.time() - start, 5)
print(f'Продолжительность подбора модели {selection_time} c.')

rmse_train_lgbm = lgbm_model.best_score_
print('Метрика RMSE для лучшей модели:\n', -rmse_train_lgbm)

result = pd.DataFrame(lgbm_model.cv_results_).sort_values('rank_test_score')
result[['rank_test_score', 'param_models', 'mean_test_score','params']].head(5)

Перейдем к подбору последней - CatBoost:

In [ ]:
param_grid = [
    {
        'models': [CatBoostRegressor(verbose=0)],
        'models__learning_rate': [0,1, 0.2, 0.4, 0.6, 0.8],  
        'models__depth': [6, 8, 10, 20], 
        'models__iterations': [10, 50, 100], 
        'models__l2_leaf_reg': [1, 3] 

    }
]

start = time.time()

cat_boost_model = RandomizedSearchCV(
    pipe_final, 
    param_distributions=param_grid,
    cv=5,
    scoring='neg_root_mean_squared_error',  
    n_jobs=-1,
    random_state=RANDOM_STATE,
)


cat_boost_model.fit(X_train, y_train)

selection_time = round(time.time() - start, 5)
print(f'Продолжительность подбора модели {selection_time} c.')

rmse_train_cat = cat_boost_model.best_score_
print('Метрика RMSE для лучшей модели:\n', -rmse_train_cat)

result = pd.DataFrame(cat_boost_model.cv_results_).sort_values('rank_test_score')
result[['rank_test_score', 'param_models', 'mean_test_score','params']].head(5)

Настало время проверить выбранные модели на валидационной выборке.

## Анализ моделей

### Сравнение моделей

Оценим подобранные модели на валидационной выборке и сравним по метрике и времени обучения и предсказывания.

In [ ]:
models = {'decision_tree_regressor': tree_descision_model,
          'lightgbm': lgbm_model, 
          'cat_boost': cat_boost_model
         }

results = {}

for name, model in models.items():
    
    start = time.time()
    predictions = model.predict(X_valid)
    predict_time = time.time() - start
    
    rmse = mean_squared_error(y_valid, predictions) ** 0.5
    
    results[name] = {
        'train_time': model.cv_results_['mean_fit_time'][model.best_index_],
        'predict_time': predict_time,
        'rmse_train': abs(model.cv_results_['mean_test_score'][model.best_index_]),
        'rmse_test': rmse,
        'rmse_difference': abs(model.cv_results_['mean_test_score'][model.best_index_]) - rmse
    }
    
df_results = pd.DataFrame(results).T
df_results

Наименьшее время для тренировки, как и следовало ожидать, требуется для дерева решений, ближайшая модель - LGBM, затем идет  CatBoost, причем разница между самой быстрой и медленной всего 3 с.

При расчете предсказания все модели уложились в одну секунду, по скорости места распределяются также, как и при обучении.

По метрике ансамбли показывают лучшее качество, причем наилучший результат у CatBoost, но не намного хуже у LGBM. Дерево решений немного хуже предсказывает, но все равно имеет значение метрики ниже порогового (2500) значения.

Мы видим небольшой разброс в моделях LGBM и CatBoost, больше отклонений наблюдаем у дерева. 

Учитывая назначение модели для применения в приложении, нам очень важно иметь минимальное время обучения и предсказания, ну и как требования к любой прогнозной модели следует добиваться минимизации ошибки. В свете этих требований подходят все модели, выберем модель с наилучшим качеством предсказаний - CatBoost.

### Анализ лучшей модели

Теперь, когда выбрали лучшую модель проверим ее на тестовой выборке и проанализируем остатки.

In [ ]:
def analyze_residuals(y_test, predictions):
    
    ''' Функция строит гистограмму распределения остатков и график распределения остатков.
        На входе принимает тестовый и расчетный целевой показатели.'''
    print('Анализ остатков')
    print('=' * 80)
    residuals = y_test - predictions

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
    axes[0].hist(residuals)
    axes[0].grid(True)
    axes[0].set_title('Гистограмма распределения остатков', fontsize=16)
    axes[0].set_xlabel('Остатки')
    axes[0].set_ylabel('Количество наблюдений')

    axes[1].scatter(y=residuals, x=predictions)
    axes[1].grid(True)
    axes[1].set_xlabel('Предсказания модели')
    axes[1].set_ylabel('Остатки')
    axes[1].set_title('Анализ дисперсии', fontsize=16)
    
    plt.tight_layout()
    plt.show()

In [ ]:
pred = cat_boost_model.predict(X_test)
analyze_residuals(y_test, pred)

Видим нормальное распределение ошибок. Дисперсия остатков указывает равное соотношение завышений\ занижений цен автомобилей. Настораживает факт, что есть отрицателбные прогнозы. Возможно, связано с сильной ассиметрией целевого признака. Преобразуем его с помощью логарифма для уменьшения разброса.

In [ ]:
y_train_log = np.log1p(y_train)
cat_boost_model.fit(X_train, y_train_log)
y_pred_log = cat_boost_model.predict(X_test)
pred =np.expm1(y_pred_log)
analyze_residuals(y_test, pred)

Теперь модель делает исключительно положительные предсказания.

Посмотрим на распределние весов в выбранной модели.

In [ ]:
feature_importance = cat_boost_model.best_estimator_.named_steps['models'].get_feature_importance()
feature_names = X_train.columns

importance_df = pd.DataFrame({'feature': feature_names, 'importance': feature_importance})
importance_df = importance_df.sort_values(by='importance')

plt.figure(figsize=(10, 6))
plt.barh(importance_df['feature'], importance_df['importance'],edgecolor='black')
plt.xlabel('Степень важности')
plt.ylabel('Признаки')
plt.title('Важность признаков в CatBoost')
plt.grid()
plt.show()

Судя по графику, модель меньше всего использовала как критерий мощность, возраст авто и модель.

## Вывод

Этот проект был проведен для создания прогнозной составляющей приложения для сервиса по продаже автомобилей. Одной из функций этого приложения является прогнозирование стоимости автомобиля пользователя после введения данных о своем автомобиле. Т.к. это приложение то для удобства пользования необходимо, чтобы модель в реальном времени быстро отрабатывала: обучалась и совершала предсказание. Отсюда можно извлечь главные требования к прогнозной модели: минимизация времени обучения\предсказания, высокая точность прогнозов.

Исходные данные предсталяли собой таблицу, состоящую из 354369 строк и 16 признаков. Основными проблемами исходного датасета были пропуски, например, в столбце `repaired`, содержащем информацию о том, была ли машина в ремонте, остутствовало порядка 20% данных. Вероятно, что автовладельцы не отмечали это умышленно, однако стоимость автомобиля ремонтированного и нет отличается в 4 раза, поэтому данный признак улучшает качество предсказания. Было бы лучше для дальнейшего развития обучающей выборки, если этот признак станет обязательным в анкетировании. Также для предотвращения появления аномальных значений, например, в годе регистрации авто, можно ввести проверяющие условия при анкетировании, чтобы предотвратить ошибки и сделать данные более качественными.

Исследовательский анализ позволил нам узнать следующие полезные для выбора модели особенности данных:
+ таргет распределен ненормально;
+ количественные признаки имеют выбросы;
+ количественные признаки с целевым связаны нелинейно;
+ судя по коэффициентам корреляции наибольшую связь с таргетом имеют категориальные признаки `age` (0.68) и `model`(0.58), тем не менее численно это является слабой связью. Что касается сильной то она есть между признаками `model`-`vehicle_type`(0.9) ,  и `model`-`brand`(1), что является недопустимым в линейных моделях.

На основании этого было предпринято решение использовать нелинейные модели, устойчивые к выбросам, а именно стандартная Decision Tree Regressor и ансамбли деревьев с градиентным бустингом - LGBM и CatBoost.

После этапа подбора гиперпараметров модели были выбраны три наиболее эффективные модели, по одной на каждый тип. Все модели прошли обучение на полном предобработанном датасете. После этого были измерены время обучения и предсказания и качество прогноза оценено с помощью метрики RMSE. Самым оптимальным выбором для выполнения поставленных задач стала модель CatBoost: по скорости не сильно уступающая быстрейшей, но демострирующая наилучшие прогнозные свойства (RMSE = 1711) из выбранных моделей. 
